In [1]:
import mysql.connector as mysql
import pandas as pd

In [2]:
db = mysql.connect(host = 'localhost',user = 'root',passwd = '**********',database = 'Countries')

In [3]:
cursor = db.cursor()

In [4]:
def modificar_paises(lista):
    if type(lista) != list:
        lista = lista['Entity']
    paises_mod = []
    for pais in lista:
        for letra in pais:
            if letra == ' ' or letra == '-' or letra == '(' or letra == ')':
                index = pais.index(letra)
                pais = pais[:index] + '_' + pais[index + 1:]
            if letra == "'" or letra == ':':
                index = pais.index(letra)
                pais = pais[:index] + pais[index + 1:]
            if letra == '&':
                index = pais.index(letra)
                pais = pais[:index] + 'and' + pais[index + 1:]
        if pais not in paises_mod:
            paises_mod.append(pais)
    return paises_mod

def inserir_dados(df,tabela):
    for i in df:
        nome_data = i
    primeiro_ano = min(df['Year'])
    ultimo_ano = max(df['Year'])
    for ano in range(primeiro_ano,ultimo_ano + 1):
        atrib = [tabela.atributos[0]]
        dados = [ano]
        for i in range(len(df)):
            if ano == df.iloc[i]['Year']:
                dados.append(df.iloc[i][nome_data])
                atrib.append(df.iloc[i]['Entity'])
        atrib_mod = modificar_paises(atrib)
        tabela.inserir_dados_double(atrib_mod,dados)

In [37]:
class Tabela:
    def __init__(self,cursor,nome,list_atributos,list_tipos):
        self.atributos = list_atributos
        self.nome = nome
        self.cursor = cursor
        try:
            comando = 'SELECT * FROM ' + self.nome + ';'
            self.cursor.execute(comando)
            self.data = self.cursor.fetchall()
        except mysql.ProgrammingError:
            atributos = '('
            for i in range(len(list_atributos)):
                atributos += list_atributos[i] + ' ' + list_tipos[i] + ','
            atributos = atributos[:-1] + ')'
            comando = 'CREATE TABLE ' + nome + ' ' + atributos + ';'
            #print(comando)
            self.cursor.execute(comando)

    def inserir_dados_double(self,nomes_atributos,lista_dados):
        nomes = '('
        for nome in nomes_atributos:
            nomes += nome + ','
        nomes = nomes[:-1] + ')'
        lista = '('
        for dado in lista_dados:
            lista += str(dado) + ','
        lista = lista[:-1] + ')'
        comando = 'INSERT INTO ' + self.nome + ' ' + nomes + ' VALUES ' + lista + ';'
        self.cursor.execute(comando)
        self.select()
        
    def clear(self):
        comando = 'DELETE FROM ' + self.nome + ';'
        self.cursor.execute(comando)
        self.select()
        
    def select(self):
        comando = 'SELECT * FROM ' + self.nome + ';'
        self.cursor.execute(comando)
        self.data = self.cursor.fetchall()
        return self.data
        
    def coluna(self,nome,dicionario = True):
        comando = 'SELECT ' + nome + ' FROM ' + self.nome + ';'
        self.cursor.execute(comando)
        col = self.cursor.fetchall()
        comando = 'SELECT ' + self.atributos[0] + ' FROM ' + self.nome + ';'
        self.cursor.execute(comando)
        ind = self.cursor.fetchall()
        if dicionario:
            mapa = dict()
            for i in range(len(ind)):
                mapa[ind[i][0]] = col[i][0]
            return mapa
        else:
            lista = list()
            for i in range(len(col)):
                lista.append(col[i][0])
            return lista
    
    def get_colunas(self,lista):
        mapa = dict()
        for item in lista:
            mapa[item] = self.coluna(item)
        return mapa
    
    def get_dataframe(self):
        mapa = dict()
        for item in self.atributos:
            mapa[item] = self.coluna(item,False)
        df = pd.DataFrame(mapa)
        return df

In [26]:
df = pd.read_csv('gdp-per-capita-worldbank.csv')
df2 = pd.read_csv('share-of-population-in-extreme-poverty.csv')
df3 = pd.read_csv('children-born-per-woman.csv')

In [8]:
paises_mod = modificar_paises(df)
tipos = ['INT']
for _ in range(len(paises_mod)):
    tipos.append('DOUBLE')

atributos = ['Ano'] + paises_mod

paises_mod2 = modificar_paises(df2)
tipos2 = ['INT']
for _ in range(len(paises_mod2)):
    tipos2.append('DOUBLE')

atributos2 = ['Ano'] + paises_mod2

paises_mod3 = modificar_paises(df3)
tipos3 = ['INT']
for _ in range(len(paises_mod3)):
    tipos3.append('DOUBLE')

atributos3 = ['Ano'] + paises_mod3

In [38]:
gdp_per_capita = Tabela(cursor,'PIB_per_capita',atributos,tipos)

In [39]:
extreme_poverty = Tabela(cursor,'Extreme_poverty',atributos2,tipos2)

In [40]:
births_per_woman = Tabela(cursor,'Births_per_woman',atributos3,tipos3)

In [45]:
db.commit()